# Writing and Optimizing Fault Tolerant Chemistry Algorithms with Qualtran 

In this tutorial we will outline how one might go about writing a **new** algorithm in qualtran, using the block encoding of the chemistry Hamiltonian as an example. We will attempt to implement the most naive block encoding of the second quantized chemistry Hamiltonian
$$
H = \sum_\sigma \sum_{pq}^{N/2} T_{pq} a_{p\sigma}^\dagger a_{q\sigma}
+
\frac{1}{2}\sum_{\alpha\beta}
\sum_{pqrs}^{N/2}
V_{pqrs} a_{p\alpha}^\dagger a_{q\alpha} a_{r\beta}^\dagger a_{s\beta} ,
$$
where $a_{p\sigma}$ ($a_{q\sigma}^\dagger$) annihilate (create) an electron in the
$p$-th orbital of spin $\sigma$. The $V_{pqrs}$ tensor contains $O(N^4)$ elements and represents a bottleneck for block encoding second quantized Hamiltonians. Our goal will be to 

1. Understand how to write algorithms in bloqs, including deferring the implementation of certain unimportant details.
2. Develop a feel for the most expensive parts of an algorithm.
3. Understand some basic principles on how to optimize an algorithm, including the trade-offs which arise.
4. Compare our naive algorithm to those implemented in qualtran.

## Prepare

To begin let's write a state preparation bloq using the [PrepareOracle](../select_and_prepare.py) abstract base class, which defines the registers we **must** define.  The goal will be to prepare the state

$$
\mathrm{Prepare}|0\rangle^{\otimes 4 \log(N/2) + 2} = \sum_{\sigma}\sum_{pq} \sqrt{\frac{|T_{pq}|}{\lambda}} |\theta_{pq}\rangle|pq\sigma\rangle|000\rangle + \sum_{\alpha\beta}\sum_{pqrs}\sqrt{\frac{|V_{pqrs}|}{2\lambda}}|\theta_{pqrs}\rangle|pq\alpha\rangle|rs\beta\rangle
$$
Here the registers $p, q, r$ and $s$ are of size $\log(N/2)$ and the spin and sign ($\theta$) registers are single qubits.

In [3]:
from functools import cached_property
from typing import Tuple
from attrs import frozen
from qualtran import SelectionRegister

from qualtran.bloqs.select_and_prepare import PrepareOracle

@frozen
class PrepareSecondQuantization(PrepareOracle):

    num_spin_orb: int

    @cached_property
    def selection_registers(self) -> Tuple[SelectionRegister, ...]:
        bitsize = (self.num_spin_orb // 2 - 1).bit_length()
        return (
            SelectionRegister(name='spatial', bitsize=bitsize, shape=(4,)),
            SelectionRegister(name='spin', bitsize=1, shape=(2,)),
            SelectionRegister(name='sign', bitsize=1),
        )